In [1]:
import numpy as np
import pandas as pd
import keras
import imageio
import os
import sagemaker
import matplotlib.pyplot as plt
from keras.applications import VGG16,ResNet50
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from glob import glob
from io import BytesIO

Using TensorFlow backend.


In [2]:
import boto3,urllib.request
from sagemaker import get_execution_role

In [3]:
s3_bucket = 'narayani1'
prefix = 'reverseimagesearch'

In [4]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [5]:
rootdir = os.getcwd()

In [6]:
datadir = os.path.join(os.getcwd(),'ut-zap50k-images-square')
datadir

'/home/ec2-user/SageMaker/ut-zap50k-images-square'

In [7]:
%%time
s3 = boto3.resource('s3')
bucket = s3.Bucket(name=s3_bucket)
objs = [obj for obj in bucket.objects.filter(Prefix='reverseimagesearch/data')]

CPU times: user 9.73 s, sys: 101 ms, total: 9.83 s
Wall time: 1min 3s


In [10]:
model = VGG16(weights='imagenet',include_top=True)

W0813 21:33:31.399017 140471305000768 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 21:33:31.418917 140471305000768 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 21:33:31.421899 140471305000768 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 21:33:31.451864 140471305000768 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:39

In [28]:
def make_preds(smaller_set):
    images = [imageio.imread(BytesIO(obj.get()['Body'].read()),'jpg') for obj in smaller_set]
    ims = np.array([np.array(Image.fromarray(i).resize((224,224))) for i in images])
    out = model.predict(ims) # (500,1000) np array
    return out

In [35]:
%%time
total = len(objs) #s3
s = 500
df = pd.DataFrame()
for i in range(0,total//s+1):
    start = i*s
    fin = start + s
    if fin > total:
        fin = total
    smaller_set = [objs[j] for j in range(start,fin)]
    #call predict
    if i < 5:
        out = make_preds(smaller_set)
        df = df.append(pd.DataFrame(out))

CPU times: user 19min 21s, sys: 1.35 s, total: 19min 22s
Wall time: 11min 20s


In [36]:
df.shape

(2500, 1000)

In [37]:
df.to_csv('points2.csv',header=None,index=None)
sagemaker_session.upload_data(
                            path=os.path.join(rootdir,'points2.csv'),
                            bucket=s3_bucket,
                            key_prefix='{}/{}'.format(prefix,'points'))

's3://narayani1/reverseimagesearch/points/points2.csv'